Example for using a robot with a "Shape
=======================================


Version | Author
------------ | -------------
0.1 | Björn Hein

License is based on Creative Commons: Attribution-NonCommercial 4.0 International (CC BY-NC 4.0) (pls. check: http://creativecommons.org/licenses/by-nc/4.0/)

# 2Dof-Robot with Shape
Create a Robot with a shape and 2-Dof 

In [ ]:
from IPEnvironmentShapeRobot import CollisionCheckerShapeRobot
from IPEnvironmentShapeRobot import ShapeRobot, ShapeRobotWithOrientation
from shapely.geometry import Point, Polygon, LineString
import matplotlib.pyplot as plt

import numpy as np



# Definition of a testing environment
trapField = dict()
trapField["obs1"] = LineString([(6, 18), (6, 8), (16, 8), (16,18)]).buffer(1.0)

# define robot geometry
robot_shape = Polygon([(-0.5, -0.5), (2, -0.5), (2.0, 0.5), (-0.5, 0.5)])
shape_robot = ShapeRobot(robot_shape, limits=[[0.0, 22.0], [0.0, 22.0]])

Use an adapted CollisionChecker for the shaped robot

In [ ]:
environment = CollisionCheckerShapeRobot(trapField, shape_robot)

In [ ]:
fig_local = plt.figure(figsize=(5,5))
ax = fig_local.add_subplot(1,1,1)
environment.drawObstacles(ax)

## Testing planning

In [ ]:
from IPBasicPRM import BasicPRM
from IPVISBasicPRM import basicPRMVisualize


basicPRM = BasicPRM(environment)
basicConfig = dict()
basicConfig["radius"] = 3
basicConfig["numNodes"] = 300

start = [[1,1]]
goal  = [[10,1]]

solution = basicPRM.planPath(start, goal, basicConfig)

fig_local = plt.figure(figsize=(10,10))
ax = fig_local.add_subplot(1,1,1)
basicPRMVisualize(basicPRM,solution,ax=ax)



## Drawing the collsion space

In [ ]:
environment
myBot = environment.robot
limits = myBot.getLimits()
fig_local = plt.figure(figsize=(10,10))
ax = fig_local.add_subplot(1,1,1)
print(limits)
for x in np.linspace(limits[0][0],limits[0][1],80):
    for y in np.linspace(limits[1][0],limits[1][1],80):
        if (environment.pointInCollision([x,y])):
            # draw in blue and transparent
            ax.scatter(x,y,marker="o", color="red", alpha=0.1)
environment.drawObstacles(ax)


## Visualize C-Space in combination with W-Space

In [ ]:
fig_local = plt.figure(figsize=(10,10))
ax = fig_local.add_subplot(1,1,1)
basicPRMVisualize(basicPRM,solution,ax=ax)

## Create an animation

In [ ]:
import copy
def interpolate_line(startPos, endPos, step_l):
    steps = []
    line = np.array(endPos) - np.array(startPos)
    line_l = np.linalg.norm(line)
    step = line / line_l * step_l
    n_steps = np.floor(line_l / step_l).astype(np.int32)
    c_step = np.array(startPos, dtype=np.float32)
    for i in range(n_steps):
        steps.append(copy.deepcopy(c_step))
        c_step += step
    if not (c_step == np.array(endPos)).all():
        steps.append(np.array(endPos))
    return steps


Extract the positions from the solution path

In [ ]:
solution

In [ ]:
basicPRM.graph.nodes[solution[1]]["pos"]

In [ ]:
solution_pos = [basicPRM.graph.nodes[node]['pos'] for node in solution]
solution_pos

Create the animation:

Attention in the following code there is  

**matplotlib.rcParams['animation.embed_limit'] = 128**

If the parameter is chosen to small the animation will stop in the middle of the path.



In [ ]:
import matplotlib.animation
matplotlib.rcParams['animation.embed_limit'] = 128
from IPython.display import HTML, display

planner=basicPRM

fig_local = plt.figure(figsize=(14, 7))
ax = fig_local.add_subplot(1, 1, 1)
## get positions for solution
solution_pos = [planner.graph.nodes[node]['pos'] for node in solution]
## interpolate to obtain a smoother movement
i_solution_pos = [solution_pos[0]]
for i in range(1, len(solution_pos)):
    segment_s = solution_pos[i-1]
    segment_e = solution_pos[i]
    i_solution_pos = i_solution_pos + interpolate_line(segment_s, segment_e, 0.1)[1:]
## animate
frames = len(i_solution_pos)

r = environment.robot
workSpaceLimits = environment.robot.getLimits()

def animate(t):
    ## clear taks space figure
    ax.cla()
    ## fix figure size
    ax.set_xlim(workSpaceLimits[0])
    ax.set_ylim(workSpaceLimits[1])
    ## draw obstacles
    #environment.drawObstacles(ax)
    ## update robot position
    pos = i_solution_pos[t]
    r.setTo(pos)
    basicPRMVisualize(basicPRM,solution,ax=ax)
    

ani = matplotlib.animation.FuncAnimation(fig_local, animate, frames=frames)
html = HTML(ani.to_jshtml())
display(html)
plt.close()



## Robot with orientation
Same as before but now the robot can also rotate

In [ ]:
import math
# define robot geometry
robot_shape2 = Polygon([(-0.5, -0.5), (2, -0.5), (2.0, 0.5), (-0.5, 0.5)])
shape_robot2 = ShapeRobotWithOrientation(robot_shape2, limits=[[0.0, 22.0,], [0.0, 22.0], [-math.pi, math.pi]])
environment = CollisionCheckerShapeRobot(trapField, shape_robot2)




In [ ]:
fig_local = plt.figure(figsize=(10,10))
ax = fig_local.add_subplot(1,1,1)
environment.drawObstacles(ax)

In [ ]:
basicPRM = BasicPRM(environment)
basicConfig = dict()
basicConfig["radius"] = 3
basicConfig["numNodes"] = 300

start = [[1,1,0]]
goal  = [[18,18,math.pi/2]]

solution = basicPRM.planPath(start, goal, basicConfig)

solution

Use for drawing the basicPRMVisualizeWspace. It basically draws the obstacles with robot with current position and orientation and uses only the first two coordinates to draw the graph, omitting the value for the orientation.

In [ ]:
from IPVISBasicPRM import basicPRMVisualizeWspace
fig_local = plt.figure(figsize=(10,10))
ax = fig_local.add_subplot(1,1,1)
basicPRMVisualizeWspace(basicPRM,solution,ax=ax)

Create animation can take longer...

In [ ]:
import matplotlib.animation
matplotlib.rcParams['animation.embed_limit'] = 256
from IPython.display import HTML, display

planner=basicPRM

fig_local = plt.figure(figsize=(14, 7))
ax = fig_local.add_subplot(1, 1, 1)
## get positions for solution
solution_pos = [planner.graph.nodes[node]['pos'] for node in solution]
## interpolate to obtain a smoother movement
i_solution_pos = [solution_pos[0]]
for i in range(1, len(solution_pos)):
    segment_s = solution_pos[i-1]
    segment_e = solution_pos[i]
    i_solution_pos = i_solution_pos + interpolate_line(segment_s, segment_e, 0.1)[1:]
## animate
frames = len(i_solution_pos)

r = environment.robot
workSpaceLimits = environment.robot.getLimits()

def animate(t):
    ## clear taks space figure
    ax.cla()
    ## fix figure size
    ax.set_xlim(workSpaceLimits[0])
    ax.set_ylim(workSpaceLimits[1])
    ## draw obstacles
    #environment.drawObstacles(ax)
    ## update robot position
    pos = i_solution_pos[t]
    r.setTo(pos)
    basicPRMVisualizeWspace(basicPRM,solution,ax=ax)
    

ani = matplotlib.animation.FuncAnimation(fig_local, animate, frames=frames)
html = HTML(ani.to_jshtml())
display(html)
plt.close()

